In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras.backend as K
import tensorboard
import shutil

from datetime import datetime as dt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
BASE_PATH = "gdrive/My Drive/kaggle/"

train_csv = pd.read_csv(BASE_PATH + "train.csv")
test_csv = pd.read_csv(BASE_PATH + "test.csv")

train_csv['Patient_Week'] = train_csv['Patient'].astype(str) + '_' + train_csv['Weeks'].astype(str)

train_data = train_csv.groupby("Patient")
# print(len(train_data))

unique_patients = train_csv['Patient'].unique()
# print(unique_patients)

for patient_id in unique_patients:
    print(train_csv[train_csv['Patient']==patient_id].iloc[0])
    break

Patient             ID00007637202177411956430
Weeks                                      -4
FVC                                      2315
Percent                               58.2536
Age                                        79
Sex                                      Male
SmokingStatus                       Ex-smoker
Patient_Week     ID00007637202177411956430_-4
Name: 0, dtype: object


In [3]:
# Need to add base fvc column, base percentage
train_csv['Weeks'] = train_csv['Weeks'].astype(int)
train_csv['base_week'] = train_csv.groupby('Patient')['Weeks'].transform('min')
train_csv['weeks_passed'] = train_csv['Weeks'] - train_csv['base_week']
train_csv['base_fvc'] = 0
train_csv['base_percent'] = 0

for index in range(len(train_csv)):
    base_data = train_csv[train_csv['Patient']==train_csv.iloc[index]['Patient']].iloc[0]
    train_csv.loc[index, 'base_fvc'] = base_data['FVC']
    train_csv.loc[index, 'base_percent'] = base_data['Percent']
    
train_csv

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,Patient_Week,base_week,weeks_passed,base_fvc,base_percent
0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker,ID00007637202177411956430_-4,-4,0,2315,58.253649
1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker,ID00007637202177411956430_5,-4,9,2315,58.253649
2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker,ID00007637202177411956430_7,-4,11,2315,58.253649
3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker,ID00007637202177411956430_9,-4,13,2315,58.253649
4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker,ID00007637202177411956430_11,-4,15,2315,58.253649
...,...,...,...,...,...,...,...,...,...,...,...,...
1544,ID00426637202313170790466,13,2712,66.594637,73,Male,Never smoked,ID00426637202313170790466_13,0,13,2925,71.824968
1545,ID00426637202313170790466,19,2978,73.126412,73,Male,Never smoked,ID00426637202313170790466_19,0,19,2925,71.824968
1546,ID00426637202313170790466,31,2908,71.407524,73,Male,Never smoked,ID00426637202313170790466_31,0,31,2925,71.824968
1547,ID00426637202313170790466,43,2975,73.052745,73,Male,Never smoked,ID00426637202313170790466_43,0,43,2925,71.824968


In [4]:
# X_data = train_csv[['Sex', 'SmokingStatus']]
X_data = train_csv[['Sex', 'SmokingStatus']]
X_data = pd.get_dummies(data=X_data, drop_first=True)
selected_columns = ['Age', 'weeks_passed', 'base_fvc', 'base_percent']
X_data[selected_columns] = train_csv[selected_columns]
y_data = train_csv['FVC']

X_train = X_data.sample(frac=0.8,random_state=0)
X_test = X_data.drop(X_train.index)

y_train = y_data.sample(frac=0.8,random_state=0).astype(float)
y_test = y_data.drop(y_train.index).astype(float)

train_stats = X_train.describe().transpose()

def norm(x):
  return (x - train_stats['mean']) / train_stats['std']


normalized_X_train = norm(X_train)
normalized_X_test = norm(X_test)

In [5]:
# sns.pairplot(X_train[['Age', 'weeks_passed', 'base_fvc', 'base_percent']], diag_kind="kde")

In [6]:
def laplace_log_likelihood(actual_fvc, predicted_fvc, confidence, return_values = False):
    """
    Calculates the modified Laplace Log Likelihood score for this competition.
    """
    sd_clipped = np.maximum(confidence, 70)
    delta = np.minimum(np.abs(actual_fvc - predicted_fvc), 1000)
    metric = - np.sqrt(2) * delta / sd_clipped - np.log(np.sqrt(2) * sd_clipped)

    if return_values:
        return metric
    else:
        return np.mean(metric)

In [7]:
# def log_custom_acc(y_test, y_pred):
#     sd_clipped = tf.maximum(tf.math.reduce_std(y_pred), 70)
#     delta = tf.minimum(tf.math.abs(tf.math.subtract(y_test, y_pred)), 1000)
    
#     metric = - tf.math.sqrt(2) * delta / sd_clipped - tf.math.log(np.sqrt(2) * sd_clipped)
#     return metric

def score(y_true, y_pred):
    tf.dtypes.cast(y_true, tf.float32)
    tf.dtypes.cast(y_pred, tf.float32)
    sigma = y_pred[:, 2] - y_pred[:, 0]
    fvc_pred = y_pred[:, 1]
    
    #sigma_clip = sigma + C1
    sigma_clip = tf.maximum(sigma, C1)
    delta = tf.abs(y_true[:, 0] - fvc_pred)
    delta = tf.minimum(delta, C2)
    sq2 = tf.sqrt( tf.dtypes.cast(2, dtype=tf.float32) )
    metric = (delta / sigma_clip)*sq2 + tf.math.log(sigma_clip* sq2)
    return K.mean(metric)

In [8]:
# Loss - laplace likelihood

def log_custom_loss(y_true, y_pred):
    tf.dtypes.cast(y_true, tf.float32)
    tf.dtypes.cast(y_pred, tf.float32)
    
    fvc_pred = y_pred
    sigma = tf.math.reduce_std(y_pred)
    
    sigma_clip = tf.maximum(sigma, 70)
    delta = tf.abs(y_true - fvc_pred)
    delta = tf.minimum(delta, 1000)
    sq2 = tf.sqrt( tf.dtypes.cast(2, dtype=tf.float32))
    metric = (delta / sigma_clip) * sq2 + tf.math.log(sigma_clip * sq2)
    return K.mean(metric)

  
def laplace_log_metric(y_true, y_pred):
  return -log_custom_loss(y_true, y_pred)

In [9]:
# For quantile
def tilted_loss(q,y,f):
    e = (y-f)
    return K.mean(K.maximum(q*e, (q-1)*e), axis=-1)

In [10]:
def build_model():
  model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=[len(X_train.keys())]),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)
#   quantile=0.5
  
#   loss=tf.keras.losses.MeanSquaredLogarithmicError()
#   loss=lambda y,f: tilted_loss(quantile,y,f)
    # loss=log_custom_loss

  model.compile(loss=log_custom_loss,
                optimizer=optimizer,
                metrics=[laplace_log_metric, 'mae', 'mse'])
  return model

model = build_model()

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               1024      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 13,505
Trainable params: 13,505
Non-trainable params: 0
____________________________________________________

In [12]:
# shutil.rmtree('logs')

In [13]:
%load_ext tensorboard
%tensorboard --logdir "gdrive/My Drive/kaggle/logs"

Reusing TensorBoard on port 6006 (pid 664), started 0:37:29 ago. (Use '!kill 664' to kill it.)

<IPython.core.display.Javascript object>

In [15]:
EPOCHS = 1000

time_now = dt.now().strftime("%Y%m%d_%H%M%S")
my_callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir=BASE_PATH + 'logs/'+str(time_now)),
]

history = model.fit(
  normalized_X_train, y_train,
  epochs=EPOCHS, validation_split = 0.2, verbose=1, callbacks=my_callbacks)

Epoch 1/1000
 1/31 [..............................] - ETA: 0s - loss: 24.7981 - laplace_log_metric: -24.7981 - mae: 2543.2949 - mse: 7064088.0000WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
31/31 [==============================] - 1s 18ms/step - loss: 24.7830 - laplace_log_metric: -24.7830 - mae: 2708.1580 - mse: 8017375.0000 - val_loss: 24.7981 - val_laplace_log_metric: -24.7981 - val_mae: 2650.1428 - val_mse: 7759697.5000
Epoch 2/1000
31/31 [==============================] - 0s 7ms/step - loss: 24.7827 - laplace_log_metric: -24.7828 - mae: 2707.5271 - mse: 8014285.0000 - val_loss: 24.7981 - val_laplace_log_metric: -24.7981 - val_mae: 2649.1404 - val_mse: 7755011.5000
Epoch 3/1000
31/31 [==============================] - 0s 7ms/step - loss: 24.7823 - lapla

In [16]:
def inference(model, X_test, y_test):
    y_pred = model.predict(X_test).reshape(-1)
    return y_pred, laplace_log_likelihood(y_test, y_pred, np.std(y_pred))

y_pred, metric = inference(model, normalized_X_test, y_test.tolist())
print(metric)

-7.931407321187439


In [17]:
np.std(y_pred)

512.40137

In [18]:
loss, log_laplace, mae, mse = model.evaluate(normalized_X_test, y_test, verbose=2)

10/10 - 0s - loss: 7.9699 - laplace_log_metric: -7.9677e+00 - mae: 521.0527 - mse: 437387.6562
